In [ ]:
import time
import pandas as pd
time.sleep(20)
import numpy as np
time.sleep(20)
from pprint import pprint
time.sleep(20)

from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.postprocessing import RejectOptionClassification
from aif360.detectors.mdss.ScoringFunctions import Bernoulli
from aif360.detectors.mdss.MDSS import MDSS
from aif360.detectors.mdss.generator import get_random_subset

from IPython.display import Markdown, display

ad_conversion_dataset = pd.read_csv('ad_campaign_data.csv')
ad_conversion_dataset.head()

print(f"Number of (instances, attributes) in the dataset = {ad_conversion_dataset.shape}")

print(f"Statistics of true conversions (0=no, 1=yes)")
print(ad_conversion_dataset.true_conversion.value_counts())

print(f"Statistics of predicted conversions (0=no, 1=yes)")
print(ad_conversion_dataset.predicted_conversion.value_counts())

features_4_scanning = ['college_educated','parents','homeowner','gender','age','income','area','politics','religion']

def print_report(data, subset):
    """ Utility function to pretty-print the subsets."""
    if subset:
        to_choose = ad_conversion_dataset[subset.keys()].isin(subset).all(axis = 1)
        df = ad_conversion_dataset[['true_conversion', 'predicted_conversion']][to_choose]
    else:
        for col in features_4_scanning:
            subset[col] = list(ad_conversion_dataset[col].unique())
        df = ad_conversion_dataset[['true_conversion', 'predicted_conversion']]

    true = df['true_conversion'].sum()
    pred = df['predicted_conversion'].sum()
    
    print('\033[1mSubset: \033[0m')
    pprint(subset)
    print('\033[1mSubset Size: \033[0m', len(df))
    print('\033[1mTrue Clicks: \033[0m', true)
    print('\033[1mPredicted Clicks: \033[0m', pred)
    print()
    
np.random.seed(11)
random_subset = get_random_subset(ad_conversion_dataset[features_4_scanning], prob = 0.05, min_elements = 10000)
print_report(ad_conversion_dataset, random_subset)

# Bias scan
scoring_function = Bernoulli(direction='negative')
scanner = MDSS(scoring_function)
 
scanned_subset, _ = scanner.scan(ad_conversion_dataset[features_4_scanning], 
                        expectations = ad_conversion_dataset['predicted_conversion'],
                        outcomes = ad_conversion_dataset['true_conversion'], 
                        penalty = 1, 
                        num_iters = 1,
                        verbose = False)

print_report(ad_conversion_dataset, scanned_subset)

print_report(ad_conversion_dataset, {'homeowner':[0]})

print_report(ad_conversion_dataset, {'homeowner':[1]})

def convert_to_standard_dataset(df, target_label_name, scores_name=""):

    # List of names corresponding to protected attribute columns in the dataset.
    # Note that the terminology "protected attribute" used in AI Fairness 360 to
    # divide the dataset into multiple groups for measuring and mitigating 
    # group-level bias.
    protected_attributes=['homeowner']
    
    # columns from the dataset that we want to select for this Bias study
    selected_features = ['gender', 'age', 'income', 'area', 'college_educated', 'homeowner', 
                         'parents', 'predicted_probability']
    
    # This privileged class is selected based on MDSS subgroup evaluation.
    # in previous steps. In our case non-homeowner (homeowner=0) are considered to 
    # be privileged and homeowners (homeowner=1) are considered as unprivileged.
    privileged_classes = [[0]]   

    # Label values which are considered favorable are listed. All others are 
    # unfavorable. Label values are mapped to 1 (favorable) and 0 (unfavorable) 
    # if they are not already binary and numerical.
    favorable_target_label = [1]

    # List of column names in the DataFrame which are to be expanded into one-hot vectors.
    categorical_features = ['parents','gender','college_educated','area','income', 'age']

    # create the `StandardDataset` object
    standard_dataset = StandardDataset(df=df, label_name=target_label_name,
                                    favorable_classes=favorable_target_label,
                                    scores_name=scores_name,
                                    protected_attribute_names=protected_attributes,
                                    privileged_classes=privileged_classes,
                                    categorical_features=categorical_features,
                                    features_to_keep=selected_features)
    if scores_name=="":
        standard_dataset.scores = standard_dataset.labels.copy()
        
    return standard_dataset

# Create two StandardDataset objects - one with true conversions and one with
# predicted conversions.

# First create the predicted dataset
ad_standard_dataset_pred = convert_to_standard_dataset(ad_conversion_dataset, 
                                            target_label_name = 'predicted_conversion',
                                            scores_name='predicted_probability')

# Use this to create the original dataset
ad_standard_dataset_orig = ad_standard_dataset_pred.copy()
ad_standard_dataset_orig.labels = ad_conversion_dataset["true_conversion"].values.reshape(-1, 1)
ad_standard_dataset_orig.scores = ad_conversion_dataset["true_conversion"].values.reshape(-1, 1)

# After converting dataset to Standard dataset your privileged class will always be 1 
# & the others would be 0 . If the column is already binary it doesn't convert to 0 & 1.

privileged_groups= [{'homeowner': 0}]
unprivileged_groups = [{'homeowner': 1}]

metric_orig = BinaryLabelDatasetMetric(ad_standard_dataset_orig, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print(f"Disparate impact for the original dataset = {metric_orig.disparate_impact():.4f}")

metric_pred = BinaryLabelDatasetMetric(ad_standard_dataset_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print(f"Disparate impact for the predicted dataset = {metric_pred.disparate_impact():.4f}")

# Split the standard dataset into train, test and validation 
# (use the same random seed to ensure instances are split in the same way)
random_seed = 1001
dataset_orig_train, dataset_orig_vt = ad_standard_dataset_orig.split([0.7], 
                                                shuffle=True, seed=random_seed)
dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], 
                                                shuffle=True, seed=random_seed+1)

print(f"Original training dataset shape: {dataset_orig_train.features.shape}")
print(f"Original validation dataset shape: {dataset_orig_valid.features.shape}")
print(f"Original testing dataset shape: {dataset_orig_test.features.shape}")

dataset_pred_train, dataset_pred_vt = ad_standard_dataset_pred.split([0.7], 
                                                shuffle=True, seed=random_seed)
dataset_pred_valid, dataset_pred_test = dataset_pred_vt.split([0.5], 
                                                shuffle=True, seed=random_seed+1)

print(f"Predicted training shape: {dataset_pred_train.features.shape}")
print(f"Predicted validation shape: {dataset_pred_valid.features.shape}")
print(f"Predicted testing shape: {dataset_pred_test.features.shape}")

# print out some labels, names, etc. for the predicted dataset
display(Markdown("#### Training Dataset shape"))
print(dataset_pred_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_pred_train.favorable_label, dataset_pred_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_pred_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_pred_train.privileged_protected_attributes, 
      dataset_pred_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_pred_train.feature_names)

# Best threshold for classification only (no fairness)

num_thresh = 300
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = dataset_pred_valid.scores > class_thresh
    dataset_pred_valid.labels[fav_inds] = dataset_pred_valid.favorable_label
    dataset_pred_valid.labels[~fav_inds] = dataset_pred_valid.unfavorable_label
    
    classified_metric_valid = ClassificationMetric(dataset_orig_valid,
                                             dataset_pred_valid, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    
    ba_arr[idx] = 0.5*(classified_metric_valid.true_positive_rate()\
                       +classified_metric_valid.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

# Metric used (should be one of allowed_metrics)
metric_name = "Statistical parity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")
    
# Fit the method
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups,
                                            low_class_thresh=0.01, high_class_thresh=0.99,
                                            num_class_thresh=100, num_ROC_margin=50,
                                            metric_name=metric_name,
                                            metric_ub=metric_ub, metric_lb=metric_lb)
dataset_transf_pred_valid = ROC.fit_predict(dataset_orig_valid, dataset_pred_valid)

print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

metric_pred_valid_transf = BinaryLabelDatasetMetric(dataset_transf_pred_valid, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Postprocessed predicted validation dataset"))
print(f"Disparate impact of unprivileged vs privileged groups = {metric_pred_valid_transf.disparate_impact():.4f}")

# Metrics function
from collections import OrderedDict
from aif360.metrics import ClassificationMetric

def compute_metrics(dataset_true, dataset_pred, 
                    unprivileged_groups, privileged_groups,
                    disp = True):
    """ Compute the key metrics """
    classified_metric_pred = ClassificationMetric(dataset_true,
                                                 dataset_pred, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate()+
                                             classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()
    metrics["Theil index"] = classified_metric_pred.theil_index()
    
    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))
    
    return metrics

# Metrics for the test set
fav_inds = dataset_pred_test.scores > best_class_thresh
dataset_pred_test.labels[fav_inds] = dataset_pred_test.favorable_label
dataset_pred_test.labels[~fav_inds] = dataset_pred_test.unfavorable_label

display(Markdown("#### Test set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_test_bef = compute_metrics(dataset_orig_test, dataset_pred_test, 
                unprivileged_groups, privileged_groups)

# Metrics for the transformed test set
dataset_transf_pred_test = ROC.predict(dataset_pred_test)

display(Markdown("#### Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_pred_test, 
                unprivileged_groups, privileged_groups)

